In [4]:
import os
from diffusers import AutoPipelineForText2Image
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import load_image, make_image_grid
import torch
from PIL  import Image 

import torchvision.transforms as transforms
import numpy as np 
from torchvision.utils import save_image
import utils_img

import sys
from io import StringIO
from pytorch_lightning import seed_everything
import warnings

from diffusers import DPMSolverMultistepScheduler


import os
os.environ['CUDA_VISIBLE_DEVICES'] = '6'

model= "/hhd2/hqq/stable_signature/stable-diffusion-2-1-base"
pipeline_img2img = AutoPipelineForImage2Image.from_pretrained(
            model, torch_dtype=torch.float16, variant="fp16", use_safetensors=True
    ).to("cuda:6")

def  img2img(imgs):
    c, h,w = imgs.shape[1], imgs.shape[-2], imgs.shape[-1]
    # #抑制输出， 重定向 stdout 到 devnull
    # original_stdout = sys.stdout
    # sys.stdout = open(os.devnull, 'w')
    warnings.filterwarnings('ignore')
    
    # 以下代码引入SDXL模型
    model= "/hdd2/hqq/stable_signature/stable-diffusion-2-1-base"
    pipeline_img2img = AutoPipelineForImage2Image.from_pretrained(
            model, torch_dtype=torch.float16, variant="fp16", use_safetensors=True
    ).to("cuda")
    seed_everything(seed=20)
    pipeline_img2img.scheduler = DPMSolverMultistepScheduler.from_config(pipeline_img2img.scheduler.config)

    #pipeline_img2img.eval()
    #pipeline_text2image.model.config.image_size = imgs.shape[-1]
    prompt = "A photo"
    generator = torch.Generator("cuda").manual_seed(0)


    for i in range(imgs.shape[0]):
        img=imgs[i]
        img2 = pipeline_img2img(prompt, image=img, strength=0.8, guidance_scale=10.5,height=imgs.shape[-2],
                                width=imgs.shape[-1],generator=generator, num_inference_steps=20).images[0]

        to_tensor=transforms.ToTensor()
        img2_tensor=utils_img.normalize_rgb(to_tensor(img2))  ## c h w , 且被归一到了-1到 1区间内   ###这一行之后没有了梯度。
        img2_tensor=img2_tensor.unsqueeze(0)
        if i==0:
            output= img2_tensor
        else:
            output= torch.cat((output,img2_tensor),dim=0)
    
    #output=output.requires_grad_(True)

    return  output 


Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00, 12.25it/s]


In [5]:
from diffusers.utils import load_image, make_image_grid

image1=load_image("datasets/train_dir/000000022396.jpg")

# # 使用Pillow裁剪图像
# left, upper, right, lower = 0, 0, 256, 256  # 裁剪区域的坐标
# image1 = image1.crop((left, upper, right, lower))

train_transform = transforms.Compose([
        transforms.RandomResizedCrop(512),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #transforms.Normalize([0.5], [0.5]),
        #utils_img.normalize_rgb,
    ])

tensor1= train_transform(image1)

In [6]:
save_image(tensor1, 't1.png') 

tensor1=tensor1.unsqueeze(0)

output= img2img(tensor1)
output=output.squeeze(0)
save_image(output, 'tt.png') 

Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00, 13.47it/s]
Seed set to 20
100%|██████████| 16/16 [00:02<00:00,  7.06it/s]


In [62]:
model= "/ssd-sata1/hqq/stable_signature/stable-diffusion-2-1-base"
pipeline_img2img = AutoPipelineForImage2Image.from_pretrained(
            model, torch_dtype=torch.float16, variant="fp16", use_safetensors=True
    ).to("cuda")
    #pipeline_text2image.model.config.image_size = imgs.shape[-1]
prompt = "no bird"

Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00, 14.60it/s]
